# GR00T Probe Evaluation Notebook

This notebook provides a convenient interface to evaluate the trained GR00T probe model.

## Features:
- **Detailed Metrics**: MSE, RMSE, MAE, correlation analysis
- **Visualizations**: Training curves and prediction plots
- **Feature Type Support**: Evaluate `mean_pooled` or `last_vector` models
- **Performance Analysis**: Per-dimension and overall performance

## Requirements:
- Trained model: `probe/best_probe_model.pth` (from training)
- Processed data: `probe_training_data_60k_processed.parquet`
- Training history: `probe/training_history.pkl` (for plots)

## 🔧 Configuration

**Important**: Make sure `FEATURE_TYPE` matches the one used during training!

In [3]:
# ===== GR00T CLEAN SETUP IN COLAB =====

# Step 1: Clone repo
!git clone https://github.com/IdoXpoz/Isaac-GR00T-fork.git
%cd Isaac-GR00T-fork

Cloning into 'Isaac-GR00T-fork'...
remote: Enumerating objects: 763, done.
remote: Counting objects: 100% (405/405), done.
remote: Compressing objects: 100% (228/228), done.
remote: Total 763 (delta 285), reused 197 (delta 177), pack-reused 358 (from 3)
Receiving objects: 100% (763/763), 48.61 MiB | 26.07 MiB/s, done.
Resolving deltas: 100% (397/397), done.
/content/Isaac-GR00T-fork


In [4]:
!git fetch
!git checkout main


Already on 'main'
Your branch is up to date with 'origin/main'.


In [5]:
!git pull origin main

From https://github.com/IdoXpoz/Isaac-GR00T-fork
 * branch            main       -> FETCH_HEAD
Already up to date.


In [6]:
# Mount Google Drive for persistent storage
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Import and run evaluation
import sys
import os

# Add current directory to path
sys.path.append(os.getcwd())

# Import evaluation function
from probe.evaluate_probe import evaluate_all_probes_for_single_action_step

print("🔍 Starting probe evaluation...")
print("=" * 60)

# Run evaluation with specified parameters
evaluate_all_probes_for_single_action_step(
    action_step=0
)

print("=" * 60)
print("🎉 Evaluation completed!")

ModuleNotFoundError: No module named 'train_probe'